ModuleNotFoundError: No module named 'qiskit_machine_learning'

In [9]:
import sys
print(sys.executable)
!{sys.executable} -m pip install qiskit-machine-learning

d:\ANACONDA\envs\qiskit\python.exe
  Using cached qiskit_machine_learning-0.8.3-py3-none-any.whl.metadata (13 kB)
  Using cached qiskit-1.4.3-cp39-abi3-win_amd64.whl.metadata (12 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached qiskit_machine_learning-0.8.3-py3-none-any.whl (231 kB)
Using cached qiskit-1.4.3-cp39-abi3-win_amd64.whl (6.5 MB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/10.7 MB 2.2 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/10.7 MB 2.3 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/10.7 MB 2.3 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/10.7 MB 2.2 MB/s eta 0:00:04
   ------------ ----------------

  You can safely remove it manually.


In [17]:
import sys
from qiskit import QuantumCircuit
from qiskit_machine_learning.connectors.torch_connector import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Estimator

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from encode import get_fm
from measure import evaluate

x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

#create pytorch dataset 
train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32),
                              torch.tensor(np.array(y_train), dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(x_test, dtype=torch.float32),
                             torch.tensor(np.array(y_test), dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=23, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=23)

#quantum circuit
num_qubits=x_train.shape[1]

# Combine feature map and ansatz into one circuit
feature_map = get_fm(num_qubits)
ansatz = TwoLocal(num_qubits, 'ry', 'cz', reps=1)  # or however you want to define it

# Combine into full circuit
qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

# Create EstimatorQNN
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=Estimator()
)

# Convert to Torch-compatible model
qc_model = TorchConnector(qnn)

#pytorch classifier
class QClassifier(nn.Module):
    def __init__(self,qnn):
        super().__init__()
        self.qnn=qnn

    def forward(self,x):
        return self.qnn(x)

#model definations
model=QClassifier(qc_model)

#loss and optimization
criterion = nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

#now comes the main point training 
def train(model,loader,criterion,optimizer,epochs=15):
    model.train()
    for epoch in range(epochs):
        total_loss=0
        for x_batch,y_batch in loader:
            y_batch=y_batch.view(-1,1)
            optimizer.zero_grad()
            output=model(x_batch)
            loss=criterion(output,y_batch)
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        print(f"EPOCH {epoch+1}/{epoch},Loss:{total_loss:.4f}")


train(model,train_loader,criterion,optimizer,epochs=15)

#evaluationn
def evaluate(model,loader):
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        for x_batch,y_batch in loader:
            y_batch=y_batch.view(-1,1)
            output=model(x_batch)
            preds=(torch.sigmoid(output)>0.5).float()
            correct+=(preds.view(-1)==y_batch).sum().item()
            total+=y_batch.size(0)
        accuracy=correct/total
        print(f"\n Test Accuracy:: {accuracy*100:.2f}%")


evaluate(model,test_loader)



C:\Users\sumanth\AppData\Local\Temp\ipykernel_16848\2420897883.py:48: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator=Estimator()
C:\Users\sumanth\AppData\Local\Temp\ipykernel_16848\2420897883.py:44: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


MissingOptionalLibraryError: "The 'PyTorch' library is required to use 'TorchConnector'. You can install it with 'pip install 'qiskit-machine-learning[torch]''."